In [1]:
#Libraries

#Python Libs
import sys
import os
import glob
import traceback
from datetime import datetime
import time
from geopy import distance


#Data Analysis Libs
import pandas as pd
import numpy as np

In [2]:
#Functions
def select_input_files(enh_buste_base_path,init_date,fin_date,suffix):
        selected_files = []
        all_files = glob.glob(os.path.join(enh_buste_base_path,"*"))

        for file_ in all_files:
                try:
                        file_date = pd.to_datetime(file_.split('/')[-1],format=('%Y_%m_%d' + suffix  + '.csv'))
                        if (file_date >= init_date) and (file_date <= fin_date):
                                selected_files.append((file_,file_date))
                except:
                        continue

        return sorted(selected_files)

def get_router_id(query_date):
    INTERMEDIATE_OTP_DATE = pd.to_datetime("2017-06-30", format="%Y-%m-%d")
    router_id = ''

    if (query_date <= INTERMEDIATE_OTP_DATE):
        return 'ctba-2017-1'
    else:
        return 'ctba-2017-2'

In [3]:
#def match_terminal_boardings(selected_trips,itineraries_start):
#        terminal_boarding_origins = selected_trips[(selected_trips['o_busCode'].str.isdigit()) & (selected_trips['o_route'] != '021')]
#        matched_terminal_boardings = terminal_boarding_origins.merge(itineraries_start,
#                                                                                                                                left_on=['o_boarding_id','o_stopPointId'],
#                                                                                                                                right_on=['user_trip_id','from_parent_station'], how='inner')
#        num_matched_terminal_boardings = len(matched_terminal_boardings.drop_duplicates(subset=['cardNum','o_boarding_id']))
#        if num_matched_terminal_boardings == 0:
#            matched_perc = 0.0
#        else:
#            matched_perc = 100*(num_matched_terminal_boardings/float(len(terminal_boarding_origins)))
#        return (matched_terminal_boardings,num_matched_terminal_boardings,matched_perc)

#def match_terminal_021_boardings(selected_trips,itineraries_start):
#        terminal_021_origins = selected_trips[(selected_trips['o_busCode'].str.isdigit()) & (selected_trips['o_route'] == '021')]
#        matched_021_terminal_boardings = terminal_021_origins.merge(itineraries_start, left_on=['o_boarding_id','o_route','o_stopPointId'], right_on=['user_trip_id','route','from_parent_station'], how='inner')
#        num_matched_021_terminal_boardings = len(matched_021_terminal_boardings.drop_duplicates(subset=['cardNum','o_boarding_id']))
#        if num_matched_021_terminal_boardings == 0:
#                terminal_021_match_perc = 0.0
#        else:
#                terminal_021_match_perc = 100*(num_matched_021_terminal_boardings/float(len(terminal_021_origins)))
#        return (len(terminal_021_origins),matched_021_terminal_boardings,num_matched_021_terminal_boardings,terminal_021_match_perc)


In [4]:
#def get_candidate_itineraries_penalty_score(otp_buste_itineraries_filtered):
#        otp_buste_itineraries_penalty = otp_buste_itineraries_filtered \
#                                                        .assign(penalty = lambda x: 2*x['start_diff'].dt.total_seconds() + x['trip_duration'].dt.total_seconds() + x['match_num_transfers']*10) \
#                                                        [['cardNum','user_trip_id','itinerary_id','match_num_transfers','match_vehicle_boarding','next_origin_dist','origin_dist','start_diff','trip_duration','penalty']] \
#                                                        .sort_values(['user_trip_id','penalty'], ascending=True)
#        return otp_buste_itineraries_penalty


## Main

In [5]:
otp_suggestions_filepath = '/local/tarciso/data/otp_itineraries/2017_05_01_user_trips_100_otp_itineraries.csv'
user_trips_folderpath = '/local/tarciso/data/enhanced-buste/user_trips/'
bus_trips_folderpath = '/local/tarciso/data/enhanced-buste/bus_trips/'
gtfs_base_folderpath = '/local/tarciso/data/gtfs/'
output_folderpath = '/local/tarciso/data/test-odmat/'

In [6]:
file_date_str = otp_suggestions_filepath.split('/')[-1].split('_user_trips_')[0]
file_date = pd.to_datetime(file_date_str,format='%Y_%m_%d')
print "Processing File:", otp_suggestions_filepath

Processing File: /local/tarciso/data/otp_itineraries/2017_05_01_user_trips_100_otp_itineraries.csv


In [7]:
        # Extracting itinerary part name for later use
        itinerary_part_name = otp_suggestions_filepath.split('/')[-1].split('_')[5]
        # Read OTP Suggestions
        otp_suggestions_raw = pd.read_csv(otp_suggestions_filepath, parse_dates=['date','otp_start_time','otp_end_time'])

        if len(otp_suggestions_raw) == 0:
            print "Zero OTP suggestions found."
            print "Skipping next steps..."
            exit(0)


In [8]:
def prepare_otp_data(otp_data):
        #Fixing prefix
        otp_data.columns = otp_data.columns.str.replace('otp_','')
        otp_data = otp_data.add_prefix('otp_')
        
        #Fixing Timezone difference - when needed
        otp_data['otp_start_time'] = otp_data['otp_start_time'] - pd.Timedelta('10800 s')
        otp_data['otp_end_time'] = otp_data['otp_end_time'] - pd.Timedelta('10800 s')
        
        #Adjusting route format to have 3 numbers
        otp_data['otp_route'] = otp_data['otp_route'].astype(str)
        otp_data['otp_route'] = np.where(otp_data['otp_mode'] == 'BUS',
                            otp_data['otp_route'].astype(str).str.replace("\.0",'').str.zfill(3),
                            otp_data['otp_route'])

        return otp_data

In [9]:
        # Prepare OTP data for analysis
        otp_suggestions = prepare_otp_data(otp_suggestions_raw)

        # Read stops data
        stops_filepath = gtfs_base_folderpath + os.sep + get_router_id(file_date) + os.sep + 'stops.txt'
        stops_df = pd.read_csv(stops_filepath)

        # Adding Parent Stop data to OTP Suggestions
        stops_parent_stations = stops_df[['stop_id','parent_station']]
        otp_suggestions = otp_suggestions.merge(stops_parent_stations.add_prefix('from_'),
                                                left_on='otp_from_stop_id',
                                                right_on='from_stop_id',
                                                how='left') \
                                        .merge(stops_parent_stations.add_prefix('to_'),
                                                left_on='otp_to_stop_id',
                                                right_on='to_stop_id',
                                                how='left') \
                                        .drop(['from_stop_id','to_stop_id'], axis=1) \
                                        .rename(index=str, columns={'from_parent_station':'otp_from_parent_station',
                                                                    'to_parent_station':'otp_to_parent_station'})
        
        otp_suggestions_bus_legs = otp_suggestions[otp_suggestions['otp_mode'] == 'BUS']
        otp_suggestions_walk_legs = otp_suggestions[otp_suggestions['otp_mode'] == 'WALK']


In [10]:
otp_suggestions.head()

,otp_date,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_start_time,otp_end_time,otp_mode,otp_route,otp_from_stop_id,otp_to_stop_id,otp_duration_mins,otp_from_parent_station,otp_to_parent_station
0,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,6.450000,NaN,14508.0
1,2017-05-01,28150.0,1,2,2017-05-01 13:45:00,2017-05-01 13:45:52,WALK,nan,NaN,NaN,0.866667,NaN,NaN
2,2017-05-01,28150.0,1,3,2017-05-01 13:51:00,2017-05-01 14:01:59,BUS,303,26195.0,25753.0,10.983333,14508.0,41756.0
3,2017-05-01,28150.0,1,4,2017-05-01 14:02:00,2017-05-01 14:09:32,WALK,nan,NaN,NaN,7.533333,NaN,NaN
4,2017-05-01,28150.0,2,1,2017-05-01 13:42:58,2017-05-01 13:57:00,BUS,822,33788.0,30994.0,14.033333,NaN,14508.0


## Match Scheduled Itineraries to Observed Bus Trips

### Read and Prepare Bus Trips data

In [11]:
        # Find OTP Suggested Itineraries in BUSTE Data
        bus_trips_filepath = bus_trips_folderpath + os.sep + file_date_str + '_bus_trips.csv'
        bus_trips = pd.read_csv(bus_trips_filepath, dtype={'route': object},parse_dates=['gps_datetime']) \
                                        .sort_values(['route','busCode','tripNum','gps_datetime']) \
                                        .assign(route = lambda x: x['route'].astype(str).str.replace("\.0",'').str.zfill(3))


In [12]:
bus_trips.head()

,route,busCode,shapeId,tripNum,stopPointId,gps_datetime,distanceTraveledShape,stop_lat,stop_lon,parent_station
29435,010,BB303,1708.0,1.0,33157.0,2017-05-01 06:28:07,537.974,-25.410517,-49.276479,NaN
29436,010,BB303,1708.0,1.0,33159.0,2017-05-01 06:30:38,1141.061,-25.411726,-49.270902,NaN
29437,010,BB303,1708.0,1.0,33158.0,2017-05-01 06:31:40,1624.751,-25.415285,-49.270134,NaN
29438,010,BB303,1708.0,1.0,30150.0,2017-05-01 06:32:26,1972.077,-25.416733,-49.267863,NaN
29439,010,BB303,1708.0,1.0,28637.0,2017-05-01 06:33:11,2378.349,-25.414184,-49.265917,NaN


#### Identify Possible Matches between OTP Itineraries and Bus Trips Observed Data

In [13]:
bus_trips.dtypes

route                            object
busCode                          object
shapeId                         float64
tripNum                         float64
stopPointId                     float64
gps_datetime             datetime64[ns]
distanceTraveledShape           float64
stop_lat                        float64
stop_lon                        float64
parent_station                  float64
dtype: object

In [14]:
otp_suggestions_bus_legs.dtypes

otp_date                   datetime64[ns]
otp_user_trip_id                  float64
otp_itinerary_id                    int64
otp_leg_id                          int64
otp_start_time             datetime64[ns]
otp_end_time               datetime64[ns]
otp_mode                           object
otp_route                          object
otp_from_stop_id                  float64
otp_to_stop_id                    float64
otp_duration_mins                 float64
otp_from_parent_station           float64
otp_to_parent_station             float64
dtype: object

In [15]:
scheduled_itin_observed_od = otp_suggestions_bus_legs.merge(bus_trips.add_prefix('bt_from_'),
                                left_on=['otp_route','otp_from_stop_id'],
                                right_on=['bt_from_route','bt_from_stopPointId'],
                                how='inner') \
                                .assign(sched_obs_start_timediff = 
                                        lambda x: np.absolute(x['bt_from_gps_datetime'] - x['otp_start_time']))
scheduled_itin_observed_od = scheduled_itin_observed_od[scheduled_itin_observed_od['sched_obs_start_timediff'] <= pd.Timedelta(minutes=60)]

In [16]:
scheduled_itin_observed_od = scheduled_itin_observed_od.merge(bus_trips.add_prefix('bt_to_'),
                                left_on=['otp_route','bt_from_busCode','bt_from_tripNum','otp_to_stop_id'],
                                right_on=['bt_to_route','bt_to_busCode','bt_to_tripNum','bt_to_stopPointId'],
                                how='inner') \
                                .assign(sched_obs_end_timediff = 
                                        lambda x: np.absolute(x['bt_to_gps_datetime'] - x['otp_end_time'])) \
                                .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','sched_obs_start_timediff','sched_obs_end_timediff'])
scheduled_itin_observed_od = scheduled_itin_observed_od[scheduled_itin_observed_od['sched_obs_end_timediff'] <= pd.Timedelta(minutes=60)]

In [17]:
scheduled_itin_observed_od_full = pd.concat([scheduled_itin_observed_od,otp_suggestions_walk_legs], sort=False)

In [18]:
#bus_trips[(bus_trips['route'] == '827') & (bus_trips['stopPointId'] == 33788)].sort_values(['gps_datetime'])

In [19]:
scheduled_itin_observed_od_full.filter(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','otp_route','bt_from_busCode','bt_from_tripNum','bt_from_stopPointId','otp_start_time','bt_from_gps_datetime','sched_obs_start_timediff','bt_to_stopPointId','otp_end_time','bt_to_gps_datetime','sched_obs_end_timediff']) \
                            .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id'])

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_route,bt_from_busCode,bt_from_tripNum,bt_from_stopPointId,otp_start_time,bt_from_gps_datetime,sched_obs_start_timediff,bt_to_stopPointId,otp_end_time,bt_to_gps_datetime,sched_obs_end_timediff
3,28150.0,1,1,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:35:18,00:03:15,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,00:00:46
4,28150.0,1,1,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:35:18,00:03:15,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,00:00:46
5,28150.0,1,1,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:35:18,00:03:15,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,00:00:46
6,28150.0,1,1,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:35:18,00:03:15,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,00:00:46
15,28150.0,1,1,827,BC929,13.0,33788.0,2017-05-01 13:38:33,2017-05-01 14:22:34,00:44:01,30993.0,2017-05-01 13:45:00,2017-05-01 14:30:58,00:45:58
0,28150.0,1,1,827,BC929,11.0,33788.0,2017-05-01 13:38:33,2017-05-01 12:47:49,00:50:44,30993.0,2017-05-01 13:45:00,2017-05-01 12:53:04,00:51:56
1,28150.0,1,2,nan,NaN,NaN,NaN,2017-05-01 13:45:00,NaT,NaT,NaN,2017-05-01 13:45:52,NaT,NaT
110,28150.0,1,3,303,DE722,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:02:48,00:11:48,25753.0,2017-05-01 14:01:59,2017-05-01 14:12:10,00:10:11
51,28150.0,1,3,303,DE710,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:27:30,00:23:30,25753.0,2017-05-01 14:01:59,2017-05-01 13:35:59,00:26:00
67,28150.0,1,3,303,DE713,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:16:34,00:34:26,25753.0,2017-05-01 14:01:59,2017-05-01 13:24:23,00:37:36


### Read and Prepare Origin/Next-Origin Pairs data

In [20]:
def compatible_dates(otp_data,ticketing_data):
        otp_date = otp_data['otp_date'].iloc[0]
        ticketing_date = pd.to_datetime(ticketing_data['o_boarding_datetime'].dt.strftime('%Y-%m-%d')[0])

        return (otp_date == ticketing_date,otp_date,ticketing_date)


In [21]:
        # Read Origin/Next-Origin Pairs for the same date
        trips_origins_filepath = user_trips_folderpath + os.sep + file_date_str + '_user_trips.csv'
        trips_on_pairs_full = pd.read_csv(trips_origins_filepath,
                                                parse_dates=['o_boarding_datetime','o_gps_datetime','next_o_boarding_datetime','next_o_gps_datetime'])
        # Checking whether OTP and ticketing dates match
        dates_compatibility, otp_date, ticketing_date = compatible_dates(otp_suggestions,trips_on_pairs_full)
        if not dates_compatibility:
                print "ERROR: OTP date", otp_date, "does not match Ticketing data", ticketing_date
                print "Skipping current day"
                exit(1)
        
        trips_on_pairs = trips_on_pairs_full.filter(['o_boarding_id','next_o_boarding_id'])
        trips_origins = trips_on_pairs_full.filter([col for col in trips_on_pairs_full.columns if col.startswith('o_')])

In [22]:
trips_origins.head()

,o_boarding_id,o_boarding_datetime,o_gps_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_stop_lat,o_stop_lon
0,180.0,2017-05-01 04:48:42,2017-05-01 04:48:17,650,HA030,1.0,36293.0,-25.568989,-49.332253
1,40392.0,2017-05-01 15:52:39,NaT,021,08044,NaN,41791.0,-25.431951,-49.296491
2,181.0,2017-05-01 04:49:13,2017-05-01 04:49:08,650,HA030,1.0,36294.0,-25.566860,-49.332900
3,31560.0,2017-05-01 14:16:55,NaT,000,00038,NaN,26051.0,-25.447479,-49.263816
4,182.0,2017-05-01 04:49:19,2017-05-01 04:49:08,650,HA030,1.0,36294.0,-25.566860,-49.332900


In [23]:
trips_on_pairs.head()

,o_boarding_id,next_o_boarding_id
0,180.0,40392.0
1,40392.0,180.0
2,181.0,31560.0
3,31560.0,181.0
4,182.0,61724.0


In [24]:
        # Selecting trips for whom OTP suggestions were found
        selected_trips = trips_origins[trips_origins['o_boarding_id'].isin(otp_suggestions['otp_user_trip_id'])]
        num_selected_trips = len(selected_trips)


In [25]:
        # Matching all kinds of boarding events to valid OTP suggestions
        itins_first_bus_legs = otp_suggestions.query('otp_mode == \'BUS\'') \
                                .groupby(['otp_user_trip_id','otp_itinerary_id']) \
                                .first() \
                                .reset_index()

In [26]:
itins_first_bus_legs.head()

,otp_user_trip_id,otp_itinerary_id,otp_date,otp_leg_id,otp_start_time,otp_end_time,otp_mode,otp_route,otp_from_stop_id,otp_to_stop_id,otp_duration_mins,otp_from_parent_station,otp_to_parent_station
0,28150.0,1,2017-05-01,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,6.450000,14508.0,14508.0
1,28150.0,2,2017-05-01,1,2017-05-01 13:42:58,2017-05-01 13:57:00,BUS,822,33788.0,30994.0,14.033333,14508.0,14508.0
2,28150.0,3,2017-05-01,2,2017-05-01 14:03:39,2017-05-01 14:20:05,BUS,870,33980.0,35079.0,16.433333,NaN,NaN
3,28150.0,4,2017-05-01,1,2017-05-01 13:57:58,2017-05-01 14:12:00,BUS,822,33788.0,30994.0,14.033333,14508.0,14508.0
4,28150.0,5,2017-05-01,2,2017-05-01 14:10:00,2017-05-01 14:14:00,BUS,821,33973.0,30995.0,4.000000,14508.0,14508.0


In [27]:
len(itins_first_bus_legs)

2000

In [28]:
itins_first_bus_legs.otp_leg_id.describe()

count    2000.000000
mean        1.726000
std         0.446121
min         1.000000
25%         1.000000
50%         2.000000
75%         2.000000
max         2.000000
Name: otp_leg_id, dtype: float64

In [29]:
itins_first_bus_legs.dtypes

otp_user_trip_id                  float64
otp_itinerary_id                    int64
otp_date                   datetime64[ns]
otp_leg_id                          int64
otp_start_time             datetime64[ns]
otp_end_time               datetime64[ns]
otp_mode                           object
otp_route                          object
otp_from_stop_id                  float64
otp_to_stop_id                    float64
otp_duration_mins                 float64
otp_from_parent_station           float64
otp_to_parent_station             float64
dtype: object

## Working with Vehicle Boardings

In [30]:
selected_trips.head()

,o_boarding_id,o_boarding_datetime,o_gps_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_stop_lat,o_stop_lon
19800,28150.0,2017-05-01 13:36:25,2017-05-01 13:35:18,827,BC929,12.0,33788.0,-25.436303,-49.362221
19801,57666.0,2017-05-01 18:46:58,NaT,021,08065,NaN,41778.0,-25.436627,-49.317949
19802,28153.0,2017-05-01 13:36:28,NaT,TCJ,06003,NaN,14478.0,-25.439980,-49.221858
19803,64706.0,2017-05-01 20:22:36,2017-05-01 19:59:34,020,BB607,7.0,28132.0,-25.435878,-49.306888
19804,28156.0,2017-05-01 13:36:28,2017-05-01 13:36:21,650,HR410,5.0,36299.0,-25.565199,-49.333825


In [31]:
itins_first_bus_legs.head()

,otp_user_trip_id,otp_itinerary_id,otp_date,otp_leg_id,otp_start_time,otp_end_time,otp_mode,otp_route,otp_from_stop_id,otp_to_stop_id,otp_duration_mins,otp_from_parent_station,otp_to_parent_station
0,28150.0,1,2017-05-01,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,6.450000,14508.0,14508.0
1,28150.0,2,2017-05-01,1,2017-05-01 13:42:58,2017-05-01 13:57:00,BUS,822,33788.0,30994.0,14.033333,14508.0,14508.0
2,28150.0,3,2017-05-01,2,2017-05-01 14:03:39,2017-05-01 14:20:05,BUS,870,33980.0,35079.0,16.433333,NaN,NaN
3,28150.0,4,2017-05-01,1,2017-05-01 13:57:58,2017-05-01 14:12:00,BUS,822,33788.0,30994.0,14.033333,14508.0,14508.0
4,28150.0,5,2017-05-01,2,2017-05-01 14:10:00,2017-05-01 14:14:00,BUS,821,33973.0,30995.0,4.000000,14508.0,14508.0


In [32]:
def match_vehicle_boardings(selected_trips,itineraries_st):
        vehicle_boarding_origins = selected_trips[np.logical_not(selected_trips['o_busCode'].str.isdigit())]
        matched_vehicle_boardings = vehicle_boarding_origins.merge(itineraries_st, left_on=['o_boarding_id','o_route','o_stopPointId'], 
                                                                   right_on=['otp_user_trip_id','otp_route','otp_from_stop_id'], how='inner')
        num_matched_vehicle_boardings = len(matched_vehicle_boardings.drop_duplicates(subset=['o_boarding_id']))
        if num_matched_vehicle_boardings == 0:
            match_perc = 0.0
        else:
            match_perc = 100*(num_matched_vehicle_boardings/float(len(vehicle_boarding_origins)))
        return (matched_vehicle_boardings,num_matched_vehicle_boardings,match_perc)

In [33]:
        # Matching vehicle boarding origins
        matched_vehicle_boardings,num_matched_vehicle_boardings,vehicle_match_perc = match_vehicle_boardings(selected_trips,itins_first_bus_legs)
        print "Vehicle boardings with matching OTP suggestions: ", num_matched_vehicle_boardings, "(",vehicle_match_perc, "%)"

        total_num_matches = num_matched_vehicle_boardings
        print "Total number of matches: ", total_num_matches, "(", 100*(total_num_matches/float(num_selected_trips)), "%)"

        if total_num_matches == 0:
            print "No match was found. Skipping next steps..."
            exit(0)


Vehicle boardings with matching OTP suggestions:  92 ( 80.701754386 %)
Total number of matches:  92 ( 46.0 %)


In [34]:
matched_vehicle_boardings.head()

,o_boarding_id,o_boarding_datetime,o_gps_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_stop_lat,o_stop_lon,otp_user_trip_id,...,otp_leg_id,otp_start_time,otp_end_time,otp_mode,otp_route,otp_from_stop_id,otp_to_stop_id,otp_duration_mins,otp_from_parent_station,otp_to_parent_station
0,28150.0,2017-05-01 13:36:25,2017-05-01 13:35:18,827,BC929,12.0,33788.0,-25.436303,-49.362221,28150.0,...,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,6.450000,14508.0,14508.0
1,28150.0,2017-05-01 13:36:25,2017-05-01 13:35:18,827,BC929,12.0,33788.0,-25.436303,-49.362221,28150.0,...,1,2017-05-01 14:28:17,2017-05-01 14:35:00,BUS,827,33788.0,30993.0,6.716667,14508.0,14508.0
2,28154.0,2017-05-01 13:36:28,2017-05-01 13:35:38,684,HA298,11.0,39378.0,-25.585230,-49.337210,28154.0,...,1,2017-05-01 13:42:13,2017-05-01 14:18:00,BUS,684,39378.0,31053.0,35.783333,14494.0,14494.0
3,28154.0,2017-05-01 13:36:28,2017-05-01 13:35:38,684,HA298,11.0,39378.0,-25.585230,-49.337210,28154.0,...,1,2017-05-01 13:57:13,2017-05-01 14:33:00,BUS,684,39378.0,31053.0,35.783333,14494.0,14494.0
4,28154.0,2017-05-01 13:36:28,2017-05-01 13:35:38,684,HA298,11.0,39378.0,-25.585230,-49.337210,28154.0,...,1,2017-05-01 14:12:13,2017-05-01 14:48:00,BUS,684,39378.0,31053.0,35.783333,14494.0,14494.0


In [35]:
vehic_first_boardings_options = matched_vehicle_boardings.merge(bus_trips, 
                                left_on=['o_route','o_busCode','o_tripNum','otp_to_stop_id'],
                               right_on=['route','busCode','tripNum','stopPointId'],
                               how='inner') \
            #.filter(['otp_user_trip_id','otp_itinerary_id','route','busCode','tripNum','stopPointId','otp_start_time','o_gps_datetime','otp_end_time','gps_datetime'])
            #.filter(selected_trips.columns)
            

In [36]:
vehic_first_boardings_options

,o_boarding_id,o_boarding_datetime,o_gps_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_stop_lat,o_stop_lon,otp_user_trip_id,...,route,busCode,shapeId,tripNum,stopPointId,gps_datetime,distanceTraveledShape,stop_lat,stop_lon,parent_station
0,28150.0,2017-05-01 13:36:25,2017-05-01 13:35:18,827,BC929,12.0,33788.0,-25.436303,-49.362221,28150.0,...,827,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0
1,28150.0,2017-05-01 13:36:25,2017-05-01 13:35:18,827,BC929,12.0,33788.0,-25.436303,-49.362221,28150.0,...,827,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0
2,28262.0,2017-05-01 13:37:59,2017-05-01 13:37:57,827,BC929,12.0,33775.0,-25.438158,-49.358207,28262.0,...,827,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0
3,28262.0,2017-05-01 13:37:59,2017-05-01 13:37:57,827,BC929,12.0,33775.0,-25.438158,-49.358207,28262.0,...,827,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0
4,28424.0,2017-05-01 13:39:51,2017-05-01 13:39:12,827,BC929,12.0,33535.0,-25.440179,-49.355608,28424.0,...,827,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0
5,28424.0,2017-05-01 13:39:51,2017-05-01 13:39:12,827,BC929,12.0,33535.0,-25.440179,-49.355608,28424.0,...,827,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0
6,28154.0,2017-05-01 13:36:28,2017-05-01 13:35:38,684,HA298,11.0,39378.0,-25.585230,-49.337210,28154.0,...,684,HA298,2148.0,11.0,31053.0,2017-05-01 14:01:41,15448.084,-25.513098,-49.295303,14494.0
7,28154.0,2017-05-01 13:36:28,2017-05-01 13:35:38,684,HA298,11.0,39378.0,-25.585230,-49.337210,28154.0,...,684,HA298,2148.0,11.0,31053.0,2017-05-01 14:01:41,15448.084,-25.513098,-49.295303,14494.0
8,28154.0,2017-05-01 13:36:28,2017-05-01 13:35:38,684,HA298,11.0,39378.0,-25.585230,-49.337210,28154.0,...,684,HA298,2148.0,11.0,31053.0,2017-05-01 14:01:41,15448.084,-25.513098,-49.295303,14494.0
9,28154.0,2017-05-01 13:36:28,2017-05-01 13:35:38,684,HA298,11.0,39378.0,-25.585230,-49.337210,28154.0,...,684,HA298,2148.0,11.0,31053.0,2017-05-01 14:01:41,15448.084,-25.513098,-49.295303,14494.0


### Removing Scheduled trips whose scheduled start time is more than 30 minutes away from the actual boarding time

In [37]:
vehic_first_boardings_options['sched_actual_start_timediff'] = np.abs(vehic_first_boardings_options['otp_start_time']-vehic_first_boardings_options['o_gps_datetime'])

In [38]:
vehic_first_boardings_options \
    .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','otp_start_time']) \
    .filter(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','route','busCode','tripNum','stopPointId','otp_start_time','o_gps_datetime','otp_end_time','gps_datetime','sched_actual_start_timediff'])

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,route,busCode,tripNum,stopPointId,otp_start_time,o_gps_datetime,otp_end_time,gps_datetime,sched_actual_start_timediff
0,28150.0,1,1,827,BC929,12.0,30993.0,2017-05-01 13:38:33,2017-05-01 13:35:18,2017-05-01 13:45:00,2017-05-01 13:44:14,00:03:15
1,28150.0,8,1,827,BC929,12.0,30993.0,2017-05-01 14:28:17,2017-05-01 13:35:18,2017-05-01 14:35:00,2017-05-01 13:44:14,00:52:59
6,28154.0,1,1,684,HA298,11.0,31053.0,2017-05-01 13:42:13,2017-05-01 13:35:38,2017-05-01 14:18:00,2017-05-01 14:01:41,00:06:35
7,28154.0,2,1,684,HA298,11.0,31053.0,2017-05-01 13:57:13,2017-05-01 13:35:38,2017-05-01 14:33:00,2017-05-01 14:01:41,00:21:35
8,28154.0,6,1,684,HA298,11.0,31053.0,2017-05-01 14:12:13,2017-05-01 13:35:38,2017-05-01 14:48:00,2017-05-01 14:01:41,00:36:35
9,28154.0,7,1,684,HA298,11.0,31053.0,2017-05-01 14:27:13,2017-05-01 13:35:38,2017-05-01 15:03:00,2017-05-01 14:01:41,00:51:35
10,28154.0,8,1,684,HA298,11.0,31053.0,2017-05-01 14:42:13,2017-05-01 13:35:38,2017-05-01 15:18:00,2017-05-01 14:01:41,01:06:35
17,28154.0,9,1,684,HA298,11.0,34160.0,2017-05-01 14:57:13,2017-05-01 13:35:38,2017-05-01 15:30:02,2017-05-01 13:57:14,01:21:35
18,28154.0,10,1,684,HA298,11.0,34160.0,2017-05-01 15:12:13,2017-05-01 13:35:38,2017-05-01 15:45:02,2017-05-01 13:57:14,01:36:35
26,28161.0,4,1,050,JB301,5.0,32454.0,2017-05-01 13:44:57,2017-05-01 13:35:59,2017-05-01 13:51:00,2017-05-01 13:39:53,00:08:58


In [39]:
#vehic_first_boardings_options = vehic_first_boardings_options[vehic_first_boardings_options['sched_actual_start_timediff'] < pd.Timedelta(minutes=30)]

In [40]:
vehic_first_boardings_options  \
    .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','otp_start_time']) \
    .filter(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','route','busCode','tripNum','stopPointId','otp_start_time','o_gps_datetime','otp_end_time','gps_datetime','sched_actual_start_timediff'])

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,route,busCode,tripNum,stopPointId,otp_start_time,o_gps_datetime,otp_end_time,gps_datetime,sched_actual_start_timediff
0,28150.0,1,1,827,BC929,12.0,30993.0,2017-05-01 13:38:33,2017-05-01 13:35:18,2017-05-01 13:45:00,2017-05-01 13:44:14,00:03:15
1,28150.0,8,1,827,BC929,12.0,30993.0,2017-05-01 14:28:17,2017-05-01 13:35:18,2017-05-01 14:35:00,2017-05-01 13:44:14,00:52:59
6,28154.0,1,1,684,HA298,11.0,31053.0,2017-05-01 13:42:13,2017-05-01 13:35:38,2017-05-01 14:18:00,2017-05-01 14:01:41,00:06:35
7,28154.0,2,1,684,HA298,11.0,31053.0,2017-05-01 13:57:13,2017-05-01 13:35:38,2017-05-01 14:33:00,2017-05-01 14:01:41,00:21:35
8,28154.0,6,1,684,HA298,11.0,31053.0,2017-05-01 14:12:13,2017-05-01 13:35:38,2017-05-01 14:48:00,2017-05-01 14:01:41,00:36:35
9,28154.0,7,1,684,HA298,11.0,31053.0,2017-05-01 14:27:13,2017-05-01 13:35:38,2017-05-01 15:03:00,2017-05-01 14:01:41,00:51:35
10,28154.0,8,1,684,HA298,11.0,31053.0,2017-05-01 14:42:13,2017-05-01 13:35:38,2017-05-01 15:18:00,2017-05-01 14:01:41,01:06:35
17,28154.0,9,1,684,HA298,11.0,34160.0,2017-05-01 14:57:13,2017-05-01 13:35:38,2017-05-01 15:30:02,2017-05-01 13:57:14,01:21:35
18,28154.0,10,1,684,HA298,11.0,34160.0,2017-05-01 15:12:13,2017-05-01 13:35:38,2017-05-01 15:45:02,2017-05-01 13:57:14,01:36:35
26,28161.0,4,1,050,JB301,5.0,32454.0,2017-05-01 13:44:57,2017-05-01 13:35:59,2017-05-01 13:51:00,2017-05-01 13:39:53,00:08:58


In [41]:
otp_suggestions.head()

,otp_date,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_start_time,otp_end_time,otp_mode,otp_route,otp_from_stop_id,otp_to_stop_id,otp_duration_mins,otp_from_parent_station,otp_to_parent_station
0,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,6.450000,NaN,14508.0
1,2017-05-01,28150.0,1,2,2017-05-01 13:45:00,2017-05-01 13:45:52,WALK,nan,NaN,NaN,0.866667,NaN,NaN
2,2017-05-01,28150.0,1,3,2017-05-01 13:51:00,2017-05-01 14:01:59,BUS,303,26195.0,25753.0,10.983333,14508.0,41756.0
3,2017-05-01,28150.0,1,4,2017-05-01 14:02:00,2017-05-01 14:09:32,WALK,nan,NaN,NaN,7.533333,NaN,NaN
4,2017-05-01,28150.0,2,1,2017-05-01 13:42:58,2017-05-01 13:57:00,BUS,822,33788.0,30994.0,14.033333,NaN,14508.0


In [42]:
scheduled_itin_observed_od_full.head()

,otp_date,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_start_time,otp_end_time,otp_mode,otp_route,otp_from_stop_id,otp_to_stop_id,...,bt_to_busCode,bt_to_shapeId,bt_to_tripNum,bt_to_stopPointId,bt_to_gps_datetime,bt_to_distanceTraveledShape,bt_to_stop_lat,bt_to_stop_lon,bt_to_parent_station,sched_obs_end_timediff
3,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0,00:00:46
4,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0,00:00:46
5,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0,00:00:46
6,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0,00:00:46
15,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,13.0,30993.0,2017-05-01 14:30:58,8568.987,-25.441705,-49.346328,14508.0,00:45:58


## Retain only OTP-BusTrips itineraries which figure in first-vehicle-boarding matched itineraries

In [43]:
matched_vehicle_boardings_itins = vehic_first_boardings_options.filter(['otp_user_trip_id','otp_itinerary_id'])

In [44]:
matched_vehicle_boardings_itins.head()

,otp_user_trip_id,otp_itinerary_id
0,28150.0,1
1,28150.0,8
2,28262.0,1
3,28262.0,9
4,28424.0,1


In [45]:
scheduled_itin_observed_od_full.head()

,otp_date,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_start_time,otp_end_time,otp_mode,otp_route,otp_from_stop_id,otp_to_stop_id,...,bt_to_busCode,bt_to_shapeId,bt_to_tripNum,bt_to_stopPointId,bt_to_gps_datetime,bt_to_distanceTraveledShape,bt_to_stop_lat,bt_to_stop_lon,bt_to_parent_station,sched_obs_end_timediff
3,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0,00:00:46
4,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0,00:00:46
5,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0,00:00:46
6,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0,00:00:46
15,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,13.0,30993.0,2017-05-01 14:30:58,8568.987,-25.441705,-49.346328,14508.0,00:45:58


In [46]:
len(scheduled_itin_observed_od_full)

19121

In [47]:
scheduled_itin_observed_od.columns

Index([u'otp_date', u'otp_user_trip_id', u'otp_itinerary_id', u'otp_leg_id',
       u'otp_start_time', u'otp_end_time', u'otp_mode', u'otp_route',
       u'otp_from_stop_id', u'otp_to_stop_id', u'otp_duration_mins',
       u'otp_from_parent_station', u'otp_to_parent_station', u'bt_from_route',
       u'bt_from_busCode', u'bt_from_shapeId', u'bt_from_tripNum',
       u'bt_from_stopPointId', u'bt_from_gps_datetime',
       u'bt_from_distanceTraveledShape', u'bt_from_stop_lat',
       u'bt_from_stop_lon', u'bt_from_parent_station',
       u'sched_obs_start_timediff', u'bt_to_route', u'bt_to_busCode',
       u'bt_to_shapeId', u'bt_to_tripNum', u'bt_to_stopPointId',
       u'bt_to_gps_datetime', u'bt_to_distanceTraveledShape',
       u'bt_to_stop_lat', u'bt_to_stop_lon', u'bt_to_parent_station',
       u'sched_obs_end_timediff'],
      dtype='object')

In [48]:
matched_vehicle_boardings_itins.columns

Index([u'otp_user_trip_id', u'otp_itinerary_id'], dtype='object')

In [49]:
vehicle_boardings_obs_sch_itin_legs = scheduled_itin_observed_od_full.merge(matched_vehicle_boardings_itins,how='inner') \
                                        .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id'])

In [50]:
len(vehicle_boardings_obs_sch_itin_legs)

4123

In [51]:
vehicle_boardings_obs_sch_itin_legs.head()

,otp_date,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_start_time,otp_end_time,otp_mode,otp_route,otp_from_stop_id,otp_to_stop_id,...,bt_to_busCode,bt_to_shapeId,bt_to_tripNum,bt_to_stopPointId,bt_to_gps_datetime,bt_to_distanceTraveledShape,bt_to_stop_lat,bt_to_stop_lon,bt_to_parent_station,sched_obs_end_timediff
0,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0,00:00:46
1,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0,00:00:46
2,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0,00:00:46
3,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,12.0,30993.0,2017-05-01 13:44:14,8568.987,-25.441705,-49.346328,14508.0,00:00:46
4,2017-05-01,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,...,BC929,2895.0,13.0,30993.0,2017-05-01 14:30:58,8568.987,-25.441705,-49.346328,14508.0,00:45:58


### Concatenating first boarding legs to other itinerary legs

#### Preparing First Boarding legs data

In [52]:
vehic_first_boardings_options.columns

Index([u'o_boarding_id', u'o_boarding_datetime', u'o_gps_datetime', u'o_route',
       u'o_busCode', u'o_tripNum', u'o_stopPointId', u'o_stop_lat',
       u'o_stop_lon', u'otp_user_trip_id', u'otp_itinerary_id', u'otp_date',
       u'otp_leg_id', u'otp_start_time', u'otp_end_time', u'otp_mode',
       u'otp_route', u'otp_from_stop_id', u'otp_to_stop_id',
       u'otp_duration_mins', u'otp_from_parent_station',
       u'otp_to_parent_station', u'route', u'busCode', u'shapeId', u'tripNum',
       u'stopPointId', u'gps_datetime', u'distanceTraveledShape', u'stop_lat',
       u'stop_lon', u'parent_station', u'sched_actual_start_timediff'],
      dtype='object')

In [53]:
vehic_first_boardings_options_clean = vehic_first_boardings_options \
    .filter(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','otp_mode','otp_route','o_busCode',
             'o_tripNum','otp_from_stop_id','otp_start_time','o_boarding_datetime','otp_to_stop_id',
             'otp_end_time','gps_datetime','otp_duration_mins']) \
    .rename(index=str, columns={'o_busCode':'bt_bus_code','o_tripNum':'bt_trip_num',
                                'o_boarding_datetime':'bt_start_time','gps_datetime':'bt_end_time'}) \
    .assign(bt_duration_mins = lambda x: (x.bt_end_time - x.bt_start_time)/pd.Timedelta(minutes=1)) \
    .assign(considered_duration_mins = lambda x: np.where(np.isnan(x.bt_duration_mins),x.otp_duration_mins,x.bt_duration_mins))

In [54]:
vehic_first_boardings_options_clean.head()

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,otp_to_stop_id,otp_end_time,bt_end_time,otp_duration_mins,bt_duration_mins,considered_duration_mins
0,28150.0,1,1,BUS,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:36:25,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,6.450000,7.816667,7.816667
1,28150.0,8,1,BUS,827,BC929,12.0,33788.0,2017-05-01 14:28:17,2017-05-01 13:36:25,30993.0,2017-05-01 14:35:00,2017-05-01 13:44:14,6.716667,7.816667,7.816667
2,28262.0,1,1,BUS,827,BC929,12.0,33775.0,2017-05-01 13:40:17,2017-05-01 13:37:59,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,4.716667,6.250000,6.250000
3,28262.0,9,1,BUS,827,BC929,12.0,33775.0,2017-05-01 14:30:05,2017-05-01 13:37:59,30993.0,2017-05-01 14:35:00,2017-05-01 13:44:14,4.916667,6.250000,6.250000
4,28424.0,1,1,BUS,827,BC929,12.0,33535.0,2017-05-01 13:41:48,2017-05-01 13:39:51,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,3.200000,4.383333,4.383333


In [55]:
len(vehic_first_boardings_options_clean)

340

#### Preparing OTP itinerary legs data

In [56]:
vehicle_boardings_obs_sch_itin_legs.columns.values

array(['otp_date', 'otp_user_trip_id', 'otp_itinerary_id', 'otp_leg_id',
       'otp_start_time', 'otp_end_time', 'otp_mode', 'otp_route',
       'otp_from_stop_id', 'otp_to_stop_id', 'otp_duration_mins',
       'otp_from_parent_station', 'otp_to_parent_station',
       'bt_from_route', 'bt_from_busCode', 'bt_from_shapeId',
       'bt_from_tripNum', 'bt_from_stopPointId', 'bt_from_gps_datetime',
       'bt_from_distanceTraveledShape', 'bt_from_stop_lat',
       'bt_from_stop_lon', 'bt_from_parent_station',
       'sched_obs_start_timediff', 'bt_to_route', 'bt_to_busCode',
       'bt_to_shapeId', 'bt_to_tripNum', 'bt_to_stopPointId',
       'bt_to_gps_datetime', 'bt_to_distanceTraveledShape',
       'bt_to_stop_lat', 'bt_to_stop_lon', 'bt_to_parent_station',
       'sched_obs_end_timediff'], dtype=object)

In [57]:
vehicle_boardings_obs_sch_itin_legs_clean = vehicle_boardings_obs_sch_itin_legs \
    .filter(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','otp_mode','otp_route','bt_from_busCode',
             'bt_from_tripNum','otp_from_stop_id','otp_start_time','bt_from_gps_datetime','otp_to_stop_id',
             'otp_end_time','bt_to_gps_datetime','otp_duration_mins'])\
    .rename(index=str, columns={'bt_from_busCode':'bt_bus_code','bt_from_tripNum':'bt_trip_num',
                                'bt_from_gps_datetime':'bt_start_time','bt_to_gps_datetime':'bt_end_time'}) \
    .assign(bt_duration_mins = lambda x: (x.bt_end_time - x.bt_start_time)/pd.Timedelta(minutes=1)) \
    .assign(considered_duration_mins = lambda x: np.where(np.isnan(x.bt_duration_mins),x.otp_duration_mins,x.bt_duration_mins))

In [58]:
vehicle_boardings_obs_sch_itin_legs_clean.head()

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,otp_to_stop_id,otp_end_time,bt_end_time,otp_duration_mins,bt_duration_mins,considered_duration_mins
0,28150.0,1,1,BUS,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:35:18,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,6.45,8.933333,8.933333
1,28150.0,1,1,BUS,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:35:18,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,6.45,8.933333,8.933333
2,28150.0,1,1,BUS,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:35:18,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,6.45,8.933333,8.933333
3,28150.0,1,1,BUS,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:35:18,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,6.45,8.933333,8.933333
4,28150.0,1,1,BUS,827,BC929,13.0,33788.0,2017-05-01 13:38:33,2017-05-01 14:22:34,30993.0,2017-05-01 13:45:00,2017-05-01 14:30:58,6.45,8.400000,8.400000


In [59]:
len(vehicle_boardings_obs_sch_itin_legs_clean)

4123

#### Concatenating Legs

In [60]:
len(vehic_first_boardings_options_clean)

340

In [61]:
vehic_first_boardings_options_clean_keys = vehic_first_boardings_options_clean.filter(['otp_user_trip_id','otp_itinerary_id','otp_leg_id']).drop_duplicates()

In [62]:
len(vehic_first_boardings_options_clean_keys)

309

In [63]:
vehic_first_boardings_options_clean_keys.head(10)

,otp_user_trip_id,otp_itinerary_id,otp_leg_id
0,28150.0,1,1
1,28150.0,8,1
2,28262.0,1,1
3,28262.0,9,1
4,28424.0,1,1
5,28424.0,10,1
6,28154.0,1,1
7,28154.0,2,1
8,28154.0,6,1
9,28154.0,7,1


In [64]:
vehicle_legs_merged = vehicle_boardings_obs_sch_itin_legs_clean.merge(vehic_first_boardings_options_clean_keys, how='outer', indicator=True)
vehicle_legs_rest_clean = vehicle_legs_merged[vehicle_legs_merged['_merge'] == 'left_only'].drop('_merge', axis=1)

In [65]:
len(vehicle_legs_rest_clean)

1728

In [66]:
vehicle_legs_rest_clean

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,otp_to_stop_id,otp_end_time,bt_end_time,otp_duration_mins,bt_duration_mins,considered_duration_mins
6,28150.0,1,2,WALK,nan,NaN,NaN,NaN,2017-05-01 13:45:00,NaT,NaN,2017-05-01 13:45:52,NaT,0.866667,NaN,0.866667
7,28150.0,1,3,BUS,303,DE722,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:02:48,25753.0,2017-05-01 14:01:59,2017-05-01 14:12:10,10.983333,9.366667,9.366667
8,28150.0,1,3,BUS,303,DE710,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:27:30,25753.0,2017-05-01 14:01:59,2017-05-01 13:35:59,10.983333,8.483333,8.483333
9,28150.0,1,3,BUS,303,DE713,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:16:34,25753.0,2017-05-01 14:01:59,2017-05-01 13:24:23,10.983333,7.816667,7.816667
10,28150.0,1,3,BUS,303,DE708,5.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:29:24,25753.0,2017-05-01 14:01:59,2017-05-01 14:37:26,10.983333,8.033333,8.033333
11,28150.0,1,3,BUS,303,DE719,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:07:22,25753.0,2017-05-01 14:01:59,2017-05-01 13:15:06,10.983333,7.733333,7.733333
12,28150.0,1,3,BUS,303,LE702,5.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:41:47,25753.0,2017-05-01 14:01:59,2017-05-01 14:50:32,10.983333,8.750000,8.750000
13,28150.0,1,4,WALK,nan,NaN,NaN,NaN,2017-05-01 14:02:00,NaT,NaN,2017-05-01 14:09:32,NaT,7.533333,NaN,7.533333
20,28150.0,8,2,WALK,nan,NaN,NaN,NaN,2017-05-01 14:35:00,NaT,NaN,2017-05-01 14:35:52,NaT,0.866667,NaN,0.866667
21,28150.0,8,3,BUS,303,LE702,5.0,26195.0,2017-05-01 14:39:00,2017-05-01 14:41:47,25753.0,2017-05-01 14:51:19,2017-05-01 14:50:32,12.316667,8.750000,8.750000


In [67]:
all_vehicle_legs_options = pd.concat([vehic_first_boardings_options_clean,vehicle_legs_rest_clean]) \
    .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','bt_start_time'])

In [68]:
len(all_vehicle_legs_options)

2068

In [69]:
all_vehicle_legs_options

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,otp_to_stop_id,otp_end_time,bt_end_time,otp_duration_mins,bt_duration_mins,considered_duration_mins
0,28150.0,1,1,BUS,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:36:25,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,6.450000,7.816667,7.816667
6,28150.0,1,2,WALK,nan,NaN,NaN,NaN,2017-05-01 13:45:00,NaT,NaN,2017-05-01 13:45:52,NaT,0.866667,NaN,0.866667
11,28150.0,1,3,BUS,303,DE719,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:07:22,25753.0,2017-05-01 14:01:59,2017-05-01 13:15:06,10.983333,7.733333,7.733333
9,28150.0,1,3,BUS,303,DE713,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:16:34,25753.0,2017-05-01 14:01:59,2017-05-01 13:24:23,10.983333,7.816667,7.816667
8,28150.0,1,3,BUS,303,DE710,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:27:30,25753.0,2017-05-01 14:01:59,2017-05-01 13:35:59,10.983333,8.483333,8.483333
7,28150.0,1,3,BUS,303,DE722,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:02:48,25753.0,2017-05-01 14:01:59,2017-05-01 14:12:10,10.983333,9.366667,9.366667
10,28150.0,1,3,BUS,303,DE708,5.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:29:24,25753.0,2017-05-01 14:01:59,2017-05-01 14:37:26,10.983333,8.033333,8.033333
12,28150.0,1,3,BUS,303,LE702,5.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:41:47,25753.0,2017-05-01 14:01:59,2017-05-01 14:50:32,10.983333,8.750000,8.750000
13,28150.0,1,4,WALK,nan,NaN,NaN,NaN,2017-05-01 14:02:00,NaT,NaN,2017-05-01 14:09:32,NaT,7.533333,NaN,7.533333
1,28150.0,8,1,BUS,827,BC929,12.0,33788.0,2017-05-01 14:28:17,2017-05-01 13:36:25,30993.0,2017-05-01 14:35:00,2017-05-01 13:44:14,6.716667,7.816667,7.816667


### Choose best actual leg matches (based on feasibility and start time)

In [70]:
sample_itinerary_options = all_vehicle_legs_options.iloc[0:9,]

In [71]:
sample_itinerary_options

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,otp_to_stop_id,otp_end_time,bt_end_time,otp_duration_mins,bt_duration_mins,considered_duration_mins
0,28150.0,1,1,BUS,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:36:25,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,6.450000,7.816667,7.816667
6,28150.0,1,2,WALK,nan,NaN,NaN,NaN,2017-05-01 13:45:00,NaT,NaN,2017-05-01 13:45:52,NaT,0.866667,NaN,0.866667
11,28150.0,1,3,BUS,303,DE719,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:07:22,25753.0,2017-05-01 14:01:59,2017-05-01 13:15:06,10.983333,7.733333,7.733333
9,28150.0,1,3,BUS,303,DE713,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:16:34,25753.0,2017-05-01 14:01:59,2017-05-01 13:24:23,10.983333,7.816667,7.816667
8,28150.0,1,3,BUS,303,DE710,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:27:30,25753.0,2017-05-01 14:01:59,2017-05-01 13:35:59,10.983333,8.483333,8.483333
7,28150.0,1,3,BUS,303,DE722,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:02:48,25753.0,2017-05-01 14:01:59,2017-05-01 14:12:10,10.983333,9.366667,9.366667
10,28150.0,1,3,BUS,303,DE708,5.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:29:24,25753.0,2017-05-01 14:01:59,2017-05-01 14:37:26,10.983333,8.033333,8.033333
12,28150.0,1,3,BUS,303,LE702,5.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:41:47,25753.0,2017-05-01 14:01:59,2017-05-01 14:50:32,10.983333,8.750000,8.750000
13,28150.0,1,4,WALK,nan,NaN,NaN,NaN,2017-05-01 14:02:00,NaT,NaN,2017-05-01 14:09:32,NaT,7.533333,NaN,7.533333


In [120]:
def choose_leg_matches(leg_matches_groups):
        colnames = leg_matches_groups.obj.columns.values
        chosen_leg_matches = pd.DataFrame(columns = colnames)
        prev_group_id = ()
        prev_itin_id = -1
        prev_leg_end_time = pd.NaT
        num_groups_not_survived = 0

        for name, group in leg_matches_groups:
            
                #print
                #print "Name:", name
                #print "Group:"
                #print group
                #print
                
                if (prev_itin_id == -1):
                        prev_itin_id = group['otp_itinerary_id'].iloc[0]
                        prev_leg_end_time = group['bt_start_time'].dt.floor('d').iloc[0]
                elif (prev_itin_id != group['otp_itinerary_id'].iloc[0]):
                        prev_leg_end_time = group['bt_start_time'].dt.floor('d').iloc[0]
                    

                #if (prev_group_id == ()):
                #        prev_leg_end_time = group['bt_start_time'].dt.floor('d')[0]

                #print
                #print "Previous leg end time:", prev_leg_end_time
                #print
                #print "Original Group"
                #print group.filter(['otp_start_time','bt_start_time','bt_end_time'])
                
                if (group['otp_mode'].iloc[0] == 'WALK'):
                    #print "Walking duration:", filtered_group['otp_duration_mins']
                    filtered_group = group.reset_index()
                    filtered_group.loc[0,'bt_start_time'] = prev_leg_end_time
                    filtered_group.loc[0,'bt_end_time'] = prev_leg_end_time + \
                        pd.Timedelta(minutes=np.rint(filtered_group['otp_duration_mins'].iloc[0]))
                    
                    #print "Filtered Group"
                    #print filtered_group
                else:
                    filtered_group = group[group['bt_start_time'] > prev_leg_end_time]
                
                #print
                #print "Filtered Group"
                #print filtered_group.filter(['otp_start_time','bt_start_time','bt_end_time'])

                if (len(filtered_group) == 0):
                        #print "Group did not survive! =("
                        num_groups_not_survived += 1
                        continue

                chosen_leg_match = filtered_group.sort_values('bt_start_time').iloc[0]
                #print "Chosen Leg"
                #print chosen_leg_match

                chosen_leg_matches = chosen_leg_matches.append(chosen_leg_match)

                #Update variables
                #prev_group_id = name
                prev_itin_id = group['otp_itinerary_id'].iloc[0]
                prev_leg_end_time = chosen_leg_match['bt_end_time']

        #print num_groups_not_survived
        return chosen_leg_matches.filter(colnames)


In [168]:
itineraries_groups = all_vehicle_legs_options.groupby(['otp_user_trip_id','otp_itinerary_id'])

In [169]:
itineraries_groups.head(10)

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,otp_to_stop_id,otp_end_time,bt_end_time,otp_duration_mins,bt_duration_mins,considered_duration_mins
0,28150.0,1,1,BUS,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:36:25,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,6.450000,7.816667,7.816667
6,28150.0,1,2,WALK,nan,NaN,NaN,NaN,2017-05-01 13:45:00,NaT,NaN,2017-05-01 13:45:52,NaT,0.866667,NaN,0.866667
11,28150.0,1,3,BUS,303,DE719,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:07:22,25753.0,2017-05-01 14:01:59,2017-05-01 13:15:06,10.983333,7.733333,7.733333
9,28150.0,1,3,BUS,303,DE713,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:16:34,25753.0,2017-05-01 14:01:59,2017-05-01 13:24:23,10.983333,7.816667,7.816667
8,28150.0,1,3,BUS,303,DE710,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 13:27:30,25753.0,2017-05-01 14:01:59,2017-05-01 13:35:59,10.983333,8.483333,8.483333
7,28150.0,1,3,BUS,303,DE722,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:02:48,25753.0,2017-05-01 14:01:59,2017-05-01 14:12:10,10.983333,9.366667,9.366667
10,28150.0,1,3,BUS,303,DE708,5.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:29:24,25753.0,2017-05-01 14:01:59,2017-05-01 14:37:26,10.983333,8.033333,8.033333
12,28150.0,1,3,BUS,303,LE702,5.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:41:47,25753.0,2017-05-01 14:01:59,2017-05-01 14:50:32,10.983333,8.750000,8.750000
13,28150.0,1,4,WALK,nan,NaN,NaN,NaN,2017-05-01 14:02:00,NaT,NaN,2017-05-01 14:09:32,NaT,7.533333,NaN,7.533333
1,28150.0,8,1,BUS,827,BC929,12.0,33788.0,2017-05-01 14:28:17,2017-05-01 13:36:25,30993.0,2017-05-01 14:35:00,2017-05-01 13:44:14,6.716667,7.816667,7.816667


In [158]:
def choose_itin_leg_matches(itin_leg_matches):
    leg_matches_groups = itin_leg_matches.groupby(['otp_leg_id'])
    colnames = leg_matches_groups.obj.columns.values
    chosen_leg_matches = pd.DataFrame(columns = colnames)
    prev_leg_id = -1
    prev_leg_end_time = pd.NaT
    num_groups_not_survived = 0

    for name, group in leg_matches_groups:

#             print
#         print "Name:", name
#             print "Group:"
#             print group
#             print

        if (prev_leg_id == -1):
            prev_leg_end_time = group['bt_start_time'].dt.floor('d').iloc[0]
        #elif (prev_itin_id != group['otp_itinerary_id'].iloc[0]):
        #        prev_leg_end_time = group['bt_start_time'].dt.floor('d').iloc[0]


            #if (prev_group_id == ()):
            #        prev_leg_end_time = group['bt_start_time'].dt.floor('d')[0]

#         print
#         print "Previous leg end time:", prev_leg_end_time
#         print
#         print "Original Group"
#         print group.filter(['otp_start_time','bt_start_time','bt_end_time'])

        if (group['otp_mode'].iloc[0] == 'WALK'):
            #print "Walking duration:", filtered_group['otp_duration_mins']
            filtered_group = group.reset_index()
            filtered_group.loc[0,'bt_start_time'] = prev_leg_end_time
            filtered_group.loc[0,'bt_end_time'] = prev_leg_end_time + \
                pd.Timedelta(minutes=np.rint(filtered_group['otp_duration_mins'].iloc[0]))

            #print "Filtered Group"
            #print filtered_group
        else:
            filtered_group = group[group['bt_start_time'] > prev_leg_end_time]

#         print
#         print "Filtered Group"
#         print filtered_group.filter(['otp_start_time','bt_start_time','bt_end_time'])

        if (len(filtered_group) == 0):
                #print "Group did not survive! =("
                num_groups_not_survived += 1
                continue

        chosen_leg_match = filtered_group.sort_values('bt_start_time').iloc[0]
#         print "Chosen Leg"
#         print chosen_leg_match

        chosen_leg_matches = chosen_leg_matches.append(chosen_leg_match)

        #Update variables
        #prev_group_id = name
        prev_leg_id = group['otp_leg_id'].iloc[0]
        prev_leg_end_time = chosen_leg_match['bt_end_time']

    #print num_groups_not_survived
    return chosen_leg_matches.filter(colnames)


In [159]:
test = itineraries_groups.apply(choose_itin_leg_matches).reset_index(drop=True)

In [160]:
test

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,otp_to_stop_id,otp_end_time,bt_end_time,otp_duration_mins,bt_duration_mins,considered_duration_mins
0,28150.0,1,1,BUS,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:36:25,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,6.450000,7.816667,7.816667
1,28150.0,1,2,WALK,nan,NaN,NaN,NaN,2017-05-01 13:45:00,2017-05-01 13:44:14,NaN,2017-05-01 13:45:52,2017-05-01 13:45:14,0.866667,NaN,0.866667
2,28150.0,1,3,BUS,303,DE722,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:02:48,25753.0,2017-05-01 14:01:59,2017-05-01 14:12:10,10.983333,9.366667,9.366667
3,28150.0,1,4,WALK,nan,NaN,NaN,NaN,2017-05-01 14:02:00,2017-05-01 14:12:10,NaN,2017-05-01 14:09:32,2017-05-01 14:20:10,7.533333,NaN,7.533333


In [161]:
legs_groups = all_vehicle_legs_options.groupby(['otp_user_trip_id','otp_itinerary_id','otp_leg_id'])

In [162]:
legs_groups.obj.columns.values

array(['otp_user_trip_id', 'otp_itinerary_id', 'otp_leg_id', 'otp_mode',
       'otp_route', 'bt_bus_code', 'bt_trip_num', 'otp_from_stop_id',
       'otp_start_time', 'bt_start_time', 'otp_to_stop_id',
       'otp_end_time', 'bt_end_time', 'otp_duration_mins',
       'bt_duration_mins', 'considered_duration_mins'], dtype=object)

In [ ]:
import time

start = time.time()
chosen_legs = choose_leg_matches(legs_groups)
#chosen_legs = itineraries_groups.apply(choose_itin_leg_matches)
end = time.time()

print "Execution time in s:", (end-start)

In [125]:
chosen_legs

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,otp_to_stop_id,otp_end_time,bt_end_time,otp_duration_mins,bt_duration_mins,considered_duration_mins
0,28150.0,1.0,1.0,BUS,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:36:25,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,6.450000,7.816667,7.816667
0,28150.0,1.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 13:45:00,2017-05-01 13:44:14,NaN,2017-05-01 13:45:52,2017-05-01 13:45:14,0.866667,NaN,0.866667
7,28150.0,1.0,3.0,BUS,303,DE722,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:02:48,25753.0,2017-05-01 14:01:59,2017-05-01 14:12:10,10.983333,9.366667,9.366667
0,28150.0,1.0,4.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:02:00,2017-05-01 14:12:10,NaN,2017-05-01 14:09:32,2017-05-01 14:20:10,7.533333,NaN,7.533333
1,28150.0,8.0,1.0,BUS,827,BC929,12.0,33788.0,2017-05-01 14:28:17,2017-05-01 13:36:25,30993.0,2017-05-01 14:35:00,2017-05-01 13:44:14,6.716667,7.816667,7.816667
0,28150.0,8.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:35:00,2017-05-01 13:44:14,NaN,2017-05-01 14:35:52,2017-05-01 13:45:14,0.866667,NaN,0.866667
24,28150.0,8.0,3.0,BUS,303,DE722,4.0,26195.0,2017-05-01 14:39:00,2017-05-01 14:02:48,25753.0,2017-05-01 14:51:19,2017-05-01 14:12:10,12.316667,9.366667,9.366667
0,28150.0,8.0,4.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:51:20,2017-05-01 14:12:10,NaN,2017-05-01 14:58:52,2017-05-01 14:20:10,7.533333,NaN,7.533333
6,28154.0,1.0,1.0,BUS,684,HA298,11.0,39378.0,2017-05-01 13:42:13,2017-05-01 13:36:28,31053.0,2017-05-01 14:18:00,2017-05-01 14:01:41,35.783333,25.216667,25.216667
0,28154.0,1.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:18:00,2017-05-01 14:01:41,NaN,2017-05-01 14:19:14,2017-05-01 14:02:41,1.233333,NaN,1.233333


#### Remove last walk legs of itinerary

In [75]:
chosen_legs_clean = chosen_legs.groupby(['otp_user_trip_id','otp_itinerary_id']) \
            .apply(lambda x: x.iloc[:-1] if x.otp_mode.iloc[-1] == 'WALK' else x) \
            .reset_index(drop=True) \
            .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id'])

In [76]:
chosen_legs_clean

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,otp_to_stop_id,otp_end_time,bt_end_time,otp_duration_mins,bt_duration_mins,considered_duration_mins
0,28150.0,1.0,1.0,BUS,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:36:25,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,6.450000,7.816667,7.816667
1,28150.0,1.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 13:45:00,2017-05-01 13:44:14,NaN,2017-05-01 13:45:52,2017-05-01 13:45:14,0.866667,NaN,0.866667
2,28150.0,1.0,3.0,BUS,303,DE722,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:02:48,25753.0,2017-05-01 14:01:59,2017-05-01 14:12:10,10.983333,9.366667,9.366667
3,28150.0,8.0,1.0,BUS,827,BC929,12.0,33788.0,2017-05-01 14:28:17,2017-05-01 13:36:25,30993.0,2017-05-01 14:35:00,2017-05-01 13:44:14,6.716667,7.816667,7.816667
4,28150.0,8.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:35:00,2017-05-01 13:44:14,NaN,2017-05-01 14:35:52,2017-05-01 13:45:14,0.866667,NaN,0.866667
5,28150.0,8.0,3.0,BUS,303,DE722,4.0,26195.0,2017-05-01 14:39:00,2017-05-01 14:02:48,25753.0,2017-05-01 14:51:19,2017-05-01 14:12:10,12.316667,9.366667,9.366667
6,28154.0,1.0,1.0,BUS,684,HA298,11.0,39378.0,2017-05-01 13:42:13,2017-05-01 13:36:28,31053.0,2017-05-01 14:18:00,2017-05-01 14:01:41,35.783333,25.216667,25.216667
7,28154.0,1.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:18:00,2017-05-01 14:01:41,NaN,2017-05-01 14:19:14,2017-05-01 14:02:41,1.233333,NaN,1.233333
8,28154.0,1.0,3.0,BUS,603,HD250,6.0,26296.0,2017-05-01 14:22:00,2017-05-01 14:07:27,25521.0,2017-05-01 14:53:48,2017-05-01 14:32:45,31.800000,25.300000,25.300000
9,28154.0,2.0,1.0,BUS,684,HA298,11.0,39378.0,2017-05-01 13:57:13,2017-05-01 13:36:28,31053.0,2017-05-01 14:33:00,2017-05-01 14:01:41,35.783333,25.216667,25.216667


#### Add stops data to legs

In [77]:
def add_stops_data_to_legs(itineraries_legs,stops_locs):
    itineraries_legs_stops = itineraries_legs.merge(stops_locs, left_on='otp_from_stop_id', right_on='stop_id', how='left') \
                                                                                .drop('stop_id', axis=1) \
                                                                                .rename(index=str, columns={'stop_lat':'from_stop_lat','stop_lon':'from_stop_lon'}) \
                                                                                .merge(stops_locations, left_on='otp_to_stop_id', right_on='stop_id', how='left') \
                                                                                .drop('stop_id', axis=1) \
                                                                                .rename(index=str, columns={'stop_lat':'to_stop_lat','stop_lon':'to_stop_lon'}) 
    return itineraries_legs_stops


In [78]:
stops_locations = stops_df[['stop_id','stop_lat','stop_lon']]

In [79]:
itineraries_legs = add_stops_data_to_legs(chosen_legs_clean,stops_locations)

In [80]:
itineraries_legs.head()

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,otp_to_stop_id,otp_end_time,bt_end_time,otp_duration_mins,bt_duration_mins,considered_duration_mins,from_stop_lat,from_stop_lon,to_stop_lat,to_stop_lon
0,28150.0,1.0,1.0,BUS,827,BC929,12.0,33788.0,2017-05-01 13:38:33,2017-05-01 13:36:25,30993.0,2017-05-01 13:45:00,2017-05-01 13:44:14,6.450000,7.816667,7.816667,-25.436303,-49.362221,-25.441705,-49.346328
1,28150.0,1.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 13:45:00,2017-05-01 13:44:14,NaN,2017-05-01 13:45:52,2017-05-01 13:45:14,0.866667,NaN,0.866667,NaN,NaN,NaN,NaN
2,28150.0,1.0,3.0,BUS,303,DE722,4.0,26195.0,2017-05-01 13:51:00,2017-05-01 14:02:48,25753.0,2017-05-01 14:01:59,2017-05-01 14:12:10,10.983333,9.366667,9.366667,-25.441720,-49.346978,-25.437968,-49.314099
3,28150.0,8.0,1.0,BUS,827,BC929,12.0,33788.0,2017-05-01 14:28:17,2017-05-01 13:36:25,30993.0,2017-05-01 14:35:00,2017-05-01 13:44:14,6.716667,7.816667,7.816667,-25.436303,-49.362221,-25.441705,-49.346328
4,28150.0,8.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:35:00,2017-05-01 13:44:14,NaN,2017-05-01 14:35:52,2017-05-01 13:45:14,0.866667,NaN,0.866667,NaN,NaN,NaN,NaN


In [81]:
itineraries_legs.columns.values

array(['otp_user_trip_id', 'otp_itinerary_id', 'otp_leg_id', 'otp_mode',
       'otp_route', 'bt_bus_code', 'bt_trip_num', 'otp_from_stop_id',
       'otp_start_time', 'bt_start_time', 'otp_to_stop_id',
       'otp_end_time', 'bt_end_time', 'otp_duration_mins',
       'bt_duration_mins', 'considered_duration_mins', 'from_stop_lat',
       'from_stop_lon', 'to_stop_lat', 'to_stop_lon'], dtype=object)

#### Add back card number

In [82]:
trips_on_pairs_full.columns

Index([u'cardNum', u'o_boarding_id', u'o_boarding_datetime', u'o_gps_datetime',
       u'o_route', u'o_busCode', u'o_tripNum', u'o_stopPointId', u'o_stop_lat',
       u'o_stop_lon', u'next_o_boarding_id', u'next_o_boarding_datetime',
       u'next_o_gps_datetime', u'next_o_route', u'next_o_busCode',
       u'next_o_tripNum', u'next_o_stopPointId', u'next_o_stop_lat',
       u'next_o_stop_lon', u'boardings_timediff', u'dist_between_origins'],
      dtype='object')

In [83]:
passenger_trips_ids = trips_on_pairs_full.filter(['cardNum','o_boarding_id']) \
                            .rename(index=str, columns={'cardNum':'card_num','o_boarding_id':'trip_id'})

In [84]:
passenger_trips_ids.head()

,card_num,trip_id
0,1886552.0,180.0
1,1886552.0,40392.0
2,3601076.0,181.0
3,3601076.0,31560.0
4,3282345.0,182.0


In [85]:
pass_itins_legs = itineraries_legs.merge(passenger_trips_ids, 
                       left_on=['otp_user_trip_id'], 
                       right_on=['trip_id'],
                       how='left') \
                .drop('otp_user_trip_id', axis=1) \
                .filter(np.append(['card_num','trip_id'],itineraries_legs.columns.values[1:])) \
                .sort_values(['card_num','trip_id','otp_itinerary_id','otp_leg_id'])

In [86]:
pass_itins_legs

,card_num,trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,...,otp_to_stop_id,otp_end_time,bt_end_time,otp_duration_mins,bt_duration_mins,considered_duration_mins,from_stop_lat,from_stop_lon,to_stop_lat,to_stop_lon
166,827182.0,28276.0,1.0,1.0,BUS,652,LA011,11.0,36694.0,2017-05-01 13:46:38,...,28609.0,2017-05-01 14:04:00,2017-05-01 13:55:44,17.366667,17.550000,17.550000,-25.534742,-49.332760,-25.511657,-49.324786
167,827182.0,28276.0,1.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:04:00,...,NaN,2017-05-01 14:04:05,2017-05-01 13:55:44,0.083333,NaN,0.083333,NaN,NaN,NaN,NaN
168,827182.0,28276.0,1.0,4.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:20:30,...,NaN,2017-05-01 14:21:55,2017-05-01 13:56:44,1.416667,NaN,1.416667,NaN,NaN,NaN,NaN
169,827182.0,28276.0,1.0,5.0,BUS,203,BD135,4.0,26199.0,2017-05-01 14:24:00,...,25732.0,2017-05-01 14:41:26,2017-05-01 14:23:46,17.433333,20.266667,20.266667,-25.491755,-49.293490,-25.448323,-49.287777
170,827182.0,28276.0,6.0,1.0,BUS,652,LA011,11.0,36694.0,2017-05-01 14:07:11,...,28609.0,2017-05-01 14:26:00,2017-05-01 13:55:44,18.816667,17.550000,17.550000,-25.534742,-49.332760,-25.511657,-49.324786
171,827182.0,28276.0,6.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:26:00,...,NaN,2017-05-01 14:26:05,2017-05-01 13:55:44,0.083333,NaN,0.083333,NaN,NaN,NaN,NaN
172,827182.0,28276.0,6.0,4.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:44:30,...,NaN,2017-05-01 14:46:29,2017-05-01 13:57:44,1.983333,NaN,1.983333,NaN,NaN,NaN,NaN
173,827182.0,28276.0,6.0,5.0,BUS,603,HD249,7.0,26294.0,2017-05-01 14:47:37,...,25732.0,2017-05-01 15:07:00,2017-05-01 14:14:50,19.383333,15.016667,15.016667,-25.492347,-49.293421,-25.448323,-49.287777
174,827182.0,28276.0,8.0,1.0,BUS,652,LA011,11.0,36694.0,2017-05-01 14:29:11,...,28609.0,2017-05-01 14:48:00,2017-05-01 13:55:44,18.816667,17.550000,17.550000,-25.534742,-49.332760,-25.511657,-49.324786
175,827182.0,28276.0,8.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:48:00,...,NaN,2017-05-01 14:48:05,2017-05-01 13:55:44,0.083333,NaN,0.083333,NaN,NaN,NaN,NaN


In [87]:
pass_itins_legs.columns.values

array(['card_num', 'trip_id', 'otp_itinerary_id', 'otp_leg_id',
       'otp_mode', 'otp_route', 'bt_bus_code', 'bt_trip_num',
       'otp_from_stop_id', 'otp_start_time', 'bt_start_time',
       'otp_to_stop_id', 'otp_end_time', 'bt_end_time',
       'otp_duration_mins', 'bt_duration_mins',
       'considered_duration_mins', 'from_stop_lat', 'from_stop_lon',
       'to_stop_lat', 'to_stop_lon'], dtype=object)

### Summarizing suggested itineraries information

In [88]:
def build_candidate_itineraries_df(chosen_leg_matches_data):
        otp_buste_itineraries = chosen_leg_matches_data \
                                        .query('otp_mode == \'BUS\'') \
                                        .groupby(['card_num','trip_id','otp_itinerary_id']) \
                                        .agg({'otp_from_stop_id': lambda x: x.iloc[0],
                                                  'bt_start_time': lambda x: x.iloc[0],
                                                  'from_stop_lat': lambda x: x.iloc[0],
                                                  'from_stop_lon': lambda x: x.iloc[0],
                                                  'otp_to_stop_id': lambda x: x.iloc[-1],
                                                  'bt_end_time': lambda x: x.iloc[-1],
                                                  'to_stop_lat': lambda x: x.iloc[-1],
                                                  'to_stop_lon': lambda x: x.iloc[-1],
                                                  'otp_mode': lambda x: len(x)}) \
                                        .rename(index=str, columns={'otp_mode':'num_transfers'}) \
                                        .reset_index() \
                                        .reindex(['card_num','trip_id','otp_itinerary_id',
                                                  'otp_from_stop_id','bt_start_time','from_stop_lat',
                                                  'from_stop_lon','otp_to_stop_id','bt_end_time',
                                                  'to_stop_lat','to_stop_lon','num_transfers'], axis=1, copy=False)\
                                        .assign(card_num = lambda x: x['card_num'].astype(float),
                                                trip_id = lambda x: x['trip_id'].astype(float),
                                                otp_itinerary_id = lambda x: x['otp_itinerary_id'].astype(float))
        return otp_buste_itineraries


In [89]:
cand_itineraries_df = build_candidate_itineraries_df(pass_itins_legs)

In [90]:
cand_itineraries_df

,card_num,trip_id,otp_itinerary_id,otp_from_stop_id,bt_start_time,from_stop_lat,from_stop_lon,otp_to_stop_id,bt_end_time,to_stop_lat,to_stop_lon,num_transfers
0,827182.0,28276.0,1.0,36694.0,2017-05-01 13:38:11,-25.534742,-49.332760,25732.0,2017-05-01 14:23:46,-25.448323,-49.287777,2
1,827182.0,28276.0,6.0,36694.0,2017-05-01 13:38:11,-25.534742,-49.332760,25732.0,2017-05-01 14:14:50,-25.448323,-49.287777,2
2,827182.0,28276.0,8.0,36694.0,2017-05-01 13:38:11,-25.534742,-49.332760,25732.0,2017-05-01 14:35:54,-25.448323,-49.287777,2
3,869122.0,28204.0,2.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,-49.350814,1
4,869122.0,28204.0,5.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,-49.350814,1
5,869122.0,28204.0,6.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,-49.350814,1
6,869122.0,28204.0,8.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,-49.350814,1
7,869122.0,28204.0,9.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,-49.350814,1
8,869122.0,28204.0,10.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,-49.350814,1
9,869122.0,52894.0,2.0,34119.0,2017-05-01 17:59:15,-25.451180,-49.350724,31000.0,2017-05-01 16:38:17,-25.441536,-49.347001,1


In [91]:
cand_itineraries_df.dtypes

card_num                   float64
trip_id                    float64
otp_itinerary_id           float64
otp_from_stop_id           float64
bt_start_time       datetime64[ns]
from_stop_lat              float64
from_stop_lon              float64
otp_to_stop_id             float64
bt_end_time         datetime64[ns]
to_stop_lat                float64
to_stop_lon                float64
num_transfers                int64
dtype: object

#### Add origin/next-origin locations

In [92]:
passenger_trips_valid_df = trips_on_pairs_full.filter(['cardNum','o_boarding_id','o_stop_lat','o_stop_lon','o_boarding_datetime','next_o_stop_lat','next_o_stop_lon','next_o_boarding_datetime']) \
                            .rename(index=str, columns={'cardNum':'card_num','o_boarding_id':'trip_id'})

In [93]:
passenger_trips_valid_df.head()

,card_num,trip_id,o_stop_lat,o_stop_lon,o_boarding_datetime,next_o_stop_lat,next_o_stop_lon,next_o_boarding_datetime
0,1886552.0,180.0,-25.568989,-49.332253,2017-05-01 04:48:42,-25.431951,-49.296491,2017-05-01 15:52:39
1,1886552.0,40392.0,-25.431951,-49.296491,2017-05-01 15:52:39,-25.568989,-49.332253,2017-05-01 04:48:42
2,3601076.0,181.0,-25.566860,-49.332900,2017-05-01 04:49:13,-25.447479,-49.263816,2017-05-01 14:16:55
3,3601076.0,31560.0,-25.447479,-49.263816,2017-05-01 14:16:55,-25.566860,-49.332900,2017-05-01 04:49:13
4,3282345.0,182.0,-25.566860,-49.332900,2017-05-01 04:49:19,-25.432534,-49.338889,2017-05-01 19:27:16


In [94]:
passenger_trips_valid_df.dtypes

card_num                           float64
trip_id                            float64
o_stop_lat                         float64
o_stop_lon                         float64
o_boarding_datetime         datetime64[ns]
next_o_stop_lat                    float64
next_o_stop_lon                    float64
next_o_boarding_datetime    datetime64[ns]
dtype: object

In [95]:
def dist(p1_lat, p1_lon, p2_lat, p2_lon):
    if(np.isnan([p1_lat, p1_lon, p2_lat, p2_lon]).any()):
        return -1
    else:
        return np.around(distance.geodesic((p1_lat,p1_lon),(p2_lat,p2_lon)).km,decimals=5)

In [96]:
def get_candidate_itineraries_summary(candidate_itineraries,trips_valid):
        otp_buste_itineraries_summary = candidate_itineraries \
                                        .merge(trips_valid,how='inner') \
                                        .assign(start_diff = lambda x: np.absolute(x['bt_start_time'] - x['o_boarding_datetime']),
                                                trip_duration = lambda x: x['bt_end_time'] - x['bt_start_time'],
                                                origin_dist = lambda y: y.apply(lambda x: dist(x['from_stop_lat'], x['from_stop_lon'], x['o_stop_lat'], x['o_stop_lon']),axis=1),
                                                next_origin_dist = lambda y: y.apply(lambda x: dist(x['to_stop_lat'], x['to_stop_lon'], x['next_o_stop_lat'], x['next_o_stop_lon']),axis=1),
                                                next_start_diff = lambda x: np.absolute(x['next_o_boarding_datetime'] - x['bt_end_time'])) \
                                        .sort_values(['card_num','trip_id'])
        return otp_buste_itineraries_summary


In [97]:
cand_itineraries_loc_validation = get_candidate_itineraries_summary(cand_itineraries_df,passenger_trips_valid_df)

In [98]:
cand_itineraries_loc_validation

,card_num,trip_id,otp_itinerary_id,otp_from_stop_id,bt_start_time,from_stop_lat,from_stop_lon,otp_to_stop_id,bt_end_time,to_stop_lat,...,o_stop_lon,o_boarding_datetime,next_o_stop_lat,next_o_stop_lon,next_o_boarding_datetime,next_origin_dist,next_start_diff,origin_dist,start_diff,trip_duration
0,827182.0,28276.0,1.0,36694.0,2017-05-01 13:38:11,-25.534742,-49.332760,25732.0,2017-05-01 14:23:46,-25.448323,...,-49.332760,2017-05-01 13:38:11,-25.448327,-49.287780,2017-05-01 22:46:43,0.00051,08:22:57,0.0,0 days,00:45:35
1,827182.0,28276.0,6.0,36694.0,2017-05-01 13:38:11,-25.534742,-49.332760,25732.0,2017-05-01 14:14:50,-25.448323,...,-49.332760,2017-05-01 13:38:11,-25.448327,-49.287780,2017-05-01 22:46:43,0.00051,08:31:53,0.0,0 days,00:36:39
2,827182.0,28276.0,8.0,36694.0,2017-05-01 13:38:11,-25.534742,-49.332760,25732.0,2017-05-01 14:35:54,-25.448323,...,-49.332760,2017-05-01 13:38:11,-25.448327,-49.287780,2017-05-01 22:46:43,0.00051,08:10:49,0.0,0 days,00:57:43
3,869122.0,28204.0,2.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
4,869122.0,28204.0,5.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
5,869122.0,28204.0,6.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
6,869122.0,28204.0,8.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
7,869122.0,28204.0,9.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
8,869122.0,28204.0,10.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
9,869122.0,52894.0,2.0,34119.0,2017-05-01 17:59:15,-25.451180,-49.350724,31000.0,2017-05-01 16:38:17,-25.441536,...,-49.350724,2017-05-01 17:59:15,-25.423548,-49.351123,2017-05-01 13:37:00,2.03535,03:01:17,0.0,0 days,-1 days +22:39:02


In [99]:
valid_candidate_itineraries = cand_itineraries_loc_validation[((cand_itineraries_loc_validation['trip_duration'] > pd.Timedelta('0s')) & 
                                                               (cand_itineraries_loc_validation['trip_duration'] < pd.Timedelta('2h'))) & 
                                                              ((cand_itineraries_loc_validation['start_diff'] >= pd.Timedelta('0s')) & 
                                                               (cand_itineraries_loc_validation['start_diff'] < pd.Timedelta('1.5h')))] \
                                    .query('origin_dist < 0.1') \
                                    .query('next_origin_dist < 2.0')
                                    #NEXT_START_DIFF
                                

In [100]:
valid_candidate_itineraries

,card_num,trip_id,otp_itinerary_id,otp_from_stop_id,bt_start_time,from_stop_lat,from_stop_lon,otp_to_stop_id,bt_end_time,to_stop_lat,...,o_stop_lon,o_boarding_datetime,next_o_stop_lat,next_o_stop_lon,next_o_boarding_datetime,next_origin_dist,next_start_diff,origin_dist,start_diff,trip_duration
0,827182.0,28276.0,1.0,36694.0,2017-05-01 13:38:11,-25.534742,-49.332760,25732.0,2017-05-01 14:23:46,-25.448323,...,-49.332760,2017-05-01 13:38:11,-25.448327,-49.287780,2017-05-01 22:46:43,0.00051,08:22:57,0.0,0 days,00:45:35
1,827182.0,28276.0,6.0,36694.0,2017-05-01 13:38:11,-25.534742,-49.332760,25732.0,2017-05-01 14:14:50,-25.448323,...,-49.332760,2017-05-01 13:38:11,-25.448327,-49.287780,2017-05-01 22:46:43,0.00051,08:31:53,0.0,0 days,00:36:39
2,827182.0,28276.0,8.0,36694.0,2017-05-01 13:38:11,-25.534742,-49.332760,25732.0,2017-05-01 14:35:54,-25.448323,...,-49.332760,2017-05-01 13:38:11,-25.448327,-49.287780,2017-05-01 22:46:43,0.00051,08:10:49,0.0,0 days,00:57:43
3,869122.0,28204.0,2.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
4,869122.0,28204.0,5.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
5,869122.0,28204.0,6.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
6,869122.0,28204.0,8.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
7,869122.0,28204.0,9.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
8,869122.0,28204.0,10.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
10,1024869.0,28174.0,1.0,33580.0,2017-05-01 13:36:42,-25.506291,-49.224019,26905.0,2017-05-01 14:22:51,-25.438600,...,-49.224019,2017-05-01 13:36:42,-25.441209,-49.275853,2017-05-01 18:30:23,0.82126,04:07:32,0.0,0 days,00:46:09


In [101]:
inferred_trip_itineraries = valid_candidate_itineraries.sort_values(['card_num','trip_id','trip_duration']) \
                                .groupby(['card_num','trip_id']) \
                                .first() \
                                .reset_index()
                        

In [102]:
inferred_trip_itineraries

,card_num,trip_id,otp_itinerary_id,otp_from_stop_id,bt_start_time,from_stop_lat,from_stop_lon,otp_to_stop_id,bt_end_time,to_stop_lat,...,o_stop_lon,o_boarding_datetime,next_o_stop_lat,next_o_stop_lon,next_o_boarding_datetime,next_origin_dist,next_start_diff,origin_dist,start_diff,trip_duration
0,827182.0,28276.0,6.0,36694.0,2017-05-01 13:38:11,-25.534742,-49.332760,25732.0,2017-05-01 14:14:50,-25.448323,...,-49.332760,2017-05-01 13:38:11,-25.448327,-49.287780,2017-05-01 22:46:43,0.00051,08:31:53,0.0,0 days,00:36:39
1,869122.0,28204.0,2.0,34260.0,2017-05-01 13:37:05,-25.423548,-49.351123,34123.0,2017-05-01 13:52:15,-25.451351,...,-49.351123,2017-05-01 13:37:05,-25.451180,-49.350724,2017-05-01 17:59:08,0.02096,04:06:53,0.0,0 days,00:15:10
2,1024869.0,28174.0,1.0,33580.0,2017-05-01 13:36:42,-25.506291,-49.224019,26905.0,2017-05-01 14:22:51,-25.438600,...,-49.224019,2017-05-01 13:36:42,-25.441209,-49.275853,2017-05-01 18:30:23,0.82126,04:07:32,0.0,0 days,00:46:09
3,1168962.0,28455.0,1.0,37403.0,2017-05-01 13:40:12,-25.557041,-49.283864,26197.0,2017-05-01 14:19:35,-25.476372,...,-49.283864,2017-05-01 13:40:12,-25.476335,-49.292629,2017-05-01 15:21:23,0.00463,01:01:48,0.0,0 days,00:39:23
4,1354462.0,28373.0,4.0,39837.0,2017-05-01 13:39:18,-25.556058,-49.332414,33987.0,2017-05-01 15:12:34,-25.428949,...,-49.332414,2017-05-01 13:39:18,-25.433111,-49.352019,2017-05-01 15:40:47,0.48663,00:28:13,0.0,0 days,01:33:16
5,1354462.0,39353.0,1.0,33745.0,2017-05-01 15:40:50,-25.435506,-49.349914,35179.0,2017-05-01 16:40:50,-25.551285,...,-49.349914,2017-05-01 15:40:50,-25.556058,-49.332414,2017-05-01 13:39:14,0.53028,03:01:36,0.0,0 days,01:00:00
6,1447119.0,58869.0,5.0,33311.0,2017-05-01 18:59:21,-25.413376,-49.301067,36283.0,2017-05-01 20:23:16,-25.573030,...,-49.301067,2017-05-01 18:59:21,-25.573303,-49.334882,2017-05-01 13:38:02,0.03086,06:45:14,0.0,0 days,01:23:55
7,1998953.0,28180.0,9.0,31854.0,2017-05-01 13:36:46,-25.366552,-49.272462,28615.0,2017-05-01 13:55:37,-25.423600,...,-49.272462,2017-05-01 13:36:46,-25.423318,-49.268572,2017-05-01 17:57:56,0.15181,04:02:19,0.0,0 days,00:18:51
8,1998953.0,52772.0,5.0,35642.0,2017-05-01 17:57:59,-25.423318,-49.268572,29384.0,2017-05-01 18:22:31,-25.366369,...,-49.268572,2017-05-01 17:57:59,-25.368502,-49.270891,2017-05-01 13:36:42,0.28449,04:45:49,0.0,0 days,00:24:32
9,2100344.0,28332.0,10.0,31691.0,2017-05-01 13:38:56,-25.381614,-49.243812,30598.0,2017-05-01 13:52:23,-25.397790,...,-49.243812,2017-05-01 13:38:56,-25.397440,-49.256690,2017-05-01 14:24:29,0.21880,00:32:06,0.0,0 days,00:13:27


In [103]:
inferred_trip_itineraries.filter(['next_origin_dist','next_start_diff','origin_dist','start_diff','trip_duration']).describe()

,next_origin_dist,next_start_diff,origin_dist,start_diff,trip_duration
count,82.000000,82,82.0,82,82
mean,0.164360,0 days 04:49:44.304878,0.0,0 days 00:00:00,0 days 00:36:20.353658
std,0.289222,0 days 02:51:32.059409,0.0,0 days 00:00:00,0 days 00:25:17.158506
min,0.000000,0 days 00:04:38,0.0,0 days 00:00:00,0 days 00:00:54
25%,0.010603,0 days 02:46:28.500000,0.0,0 days 00:00:00,0 days 00:14:55.500000
50%,0.028550,0 days 04:38:14,0.0,0 days 00:00:00,0 days 00:33:24
75%,0.185240,0 days 07:32:12.750000,0.0,0 days 00:00:00,0 days 00:47:29.250000
max,1.303900,0 days 10:03:55,0.0,0 days 00:00:00,0 days 01:41:49


#### Writing Inferred Trips Itineraries dataset to file

In [104]:
inf_trips_itineraries_output_filepath = output_folderpath + os.sep + file_date_str + '_' + itinerary_part_name + '_itins_inf_trips.csv'
inferred_trip_itineraries.to_csv(inf_trips_itineraries_output_filepath,index=False)

#### Getting Itineraries Legs back

In [105]:
    inferred_trip_itineraries_legs = pass_itins_legs.merge(inferred_trip_itineraries \
                                                           .filter(['card_num','trip_id','otp_itinerary_id']),
                                                           how='inner') \
                                                    .sort_values(['card_num','trip_id','otp_itinerary_id','otp_leg_id'])

In [106]:
inferred_trip_itineraries_legs

,card_num,trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,...,otp_to_stop_id,otp_end_time,bt_end_time,otp_duration_mins,bt_duration_mins,considered_duration_mins,from_stop_lat,from_stop_lon,to_stop_lat,to_stop_lon
0,827182.0,28276.0,6.0,1.0,BUS,652,LA011,11.0,36694.0,2017-05-01 14:07:11,...,28609.0,2017-05-01 14:26:00,2017-05-01 13:55:44,18.816667,17.550000,17.550000,-25.534742,-49.332760,-25.511657,-49.324786
1,827182.0,28276.0,6.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:26:00,...,NaN,2017-05-01 14:26:05,2017-05-01 13:55:44,0.083333,NaN,0.083333,NaN,NaN,NaN,NaN
2,827182.0,28276.0,6.0,4.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:44:30,...,NaN,2017-05-01 14:46:29,2017-05-01 13:57:44,1.983333,NaN,1.983333,NaN,NaN,NaN,NaN
3,827182.0,28276.0,6.0,5.0,BUS,603,HD249,7.0,26294.0,2017-05-01 14:47:37,...,25732.0,2017-05-01 15:07:00,2017-05-01 14:14:50,19.383333,15.016667,15.016667,-25.492347,-49.293421,-25.448323,-49.287777
4,869122.0,28204.0,2.0,1.0,BUS,040,HB604,4.0,34260.0,2017-05-01 14:07:00,...,34123.0,2017-05-01 14:22:38,2017-05-01 13:52:15,15.633333,15.166667,15.166667,-25.423548,-49.351123,-25.451351,-49.350814
5,1024869.0,28174.0,1.0,1.0,BUS,515,EA303,10.0,33580.0,2017-05-01 13:40:27,...,27791.0,2017-05-01 13:56:00,2017-05-01 13:52:12,15.550000,15.500000,15.500000,-25.506291,-49.224019,-25.481843,-49.246769
6,1024869.0,28174.0,1.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 13:56:00,...,NaN,2017-05-01 13:58:59,2017-05-01 13:55:12,2.983333,NaN,2.983333,NaN,NaN,NaN,NaN
7,1024869.0,28174.0,1.0,3.0,BUS,503,KE842,9.0,27474.0,2017-05-01 14:01:04,...,26905.0,2017-05-01 14:18:06,2017-05-01 14:22:51,17.033333,17.350000,17.350000,-25.481586,-49.247071,-25.438600,-49.268210
8,1168962.0,28455.0,1.0,1.0,BUS,639,GA142,10.0,37403.0,2017-05-01 13:38:48,...,31026.0,2017-05-01 14:00:00,2017-05-01 13:58:18,21.200000,18.100000,18.100000,-25.557041,-49.283864,-25.512969,-49.294833
9,1168962.0,28455.0,1.0,2.0,WALK,nan,NaN,NaN,NaN,2017-05-01 14:00:00,...,NaN,2017-05-01 14:04:15,2017-05-01 14:02:18,4.250000,NaN,4.250000,NaN,NaN,NaN,NaN


#### Writing Inferred Trips Itineraries Legs dataset to file

In [107]:
inf_trips_itineraries_legs_output_filepath = output_folderpath + os.sep + file_date_str + '_' + itinerary_part_name + '_legs_inf_trips.csv'
inferred_trip_itineraries_legs.to_csv(inf_trips_itineraries_legs_output_filepath,index=False)

In [108]:
#if len(pass_itins_legs) == 0:
#            print "No matches left after matching and selecting feasible bus legs."
#            print "Skipping next steps..."
#            exit(0)

In [109]:
# Writing suggested itineraries dataset to file
#actual_itineraries_output_filepath = output_folderpath + os.sep + file_date_str + '_' + itinerary_part_name + '_actual_itin.csv'
#pass_itins_legs.to_csv(actual_itineraries_output_filepath, index=False)